# Create a projection from the IOC data

There a 3 components to a projection with the poses:
1. Features from the feature table (Feature Table)
2. A Projection (Projection table)
3. Mappings of the coordinates to the projection and features (MapProjectionFeature Table)

As the keypoints and angles are already provided in the associated feature data, the atlas is not needed to create the projection.

In [ ]:
from emv.db.dao import DataAccessObject
from sqlalchemy.sql import text
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import numpy as np
from emv.api.models import Projection, MapProjectionFeatureCreate
from emv.db.queries import create_projection, create_map_projection_feature
from umap import UMAP
import numba

In [ ]:
# Retrieve the transformed features for the poses, feature_type = 'pose_image' is a smaller dataset than just 'pose'
# In comparison to the full data, it has normalized keypoints and embeddings
query = text("SELECT * FROM feature WHERE feature_type = 'pose_image'")
df = pd.DataFrame(DataAccessObject().fetch_all(query))
df['embedding_33'] = df['embedding_33'].apply(lambda x: ast.literal_eval(x))

In [ ]:
df.iloc[0]

In [ ]:
df 

In [ ]:
len(df)

In [ ]:
# default values that assume an atlas, not necessary, 
# but it's good to give the correct values when creating the projection

total_tiles = len(df) # either all features or a subset of features
atlas_width = 4096
max_tile_size = 512
max_tiles_per_atlas = (atlas_width // max_tile_size) ** 2
atlas_count = int(total_tiles / max_tiles_per_atlas) + 1

In [ ]:
# Create the projection, replace the names with the desired ones, library_id = 2 is for the IOC
projection = Projection(
    projection_name="IOC Poses Cylindrical UMAP",
    version="0.0.1",
    library_id=2,
    model_name="openpifpaf_fast",
    model_params={},
    data={},
    dimension=3,
    atlas_folder_path="",
    atlas_width=atlas_width,
    tile_size=max_tile_size,
    atlas_count=atlas_count,
    total_tiles=total_tiles,
    tiles_per_atlas=max_tiles_per_atlas,
)

projection_id = create_projection(projection)['projection_id']

In [ ]:
@numba.njit(fastmath=True)
def cylinder_euclidean_grad(x, y, cylinder_dimension=2*np.pi, linear_dimension=1.0):
    """Euclidean distance and gradient for cylindrical projection.

    x, y: Points between which the distance and gradient are computed.
    cylinder_dimension: The dimension of the cylindrical wraparound (default 2*pi).
    linear_dimension: The linear dimension (default 1.0).
    """
    distance_sqr = 0.0
    g = np.zeros_like(x)
    
    # Cylindrical dimension (e.g., angular wraparound)
    a = abs(x[0] - y[0])
    if 2 * a < cylinder_dimension:
        distance_sqr += a ** 2
        g[0] = (x[0] - y[0])
    else:
        distance_sqr += (cylinder_dimension - a) ** 2
        g[0] = (x[0] - y[0]) * (a - cylinder_dimension) / a
    
    # Linear dimension (e.g., height)
    b = abs(x[1] - y[1])
    distance_sqr += b ** 2
    g[1] = (x[1] - y[1])
    
    distance = np.sqrt(distance_sqr)
    return distance, g / (1e-6 + distance)

In [ ]:
cylinder_mapper = UMAP(output_metric=cylinder_euclidean_grad, min_dist=0.1, n_neighbors=100, random_state=42)
data = np.array(df['embedding_33'].tolist())
embedding = cylinder_mapper.fit_transform(data)

In [ ]:
df["sport"] = df.data.map(lambda x: x['sport'])
color_palette = sns.color_palette("Set2", n_colors=len(df.sport.unique()))
colors = df.sport.map(lambda x: color_palette[list(df.sport.unique()).index(x)])

In [ ]:
# Panorama dimensions
R_pano = 1
H_pano = 1

# Cylindrical dimension (theta) and height (h)
cylinder_dimension = 2 * np.pi
radius = R_pano  # Radius of the cylinder

# Extract the cylindrical (theta) and linear (h) coordinates
theta_coords = cylinder_mapper.embedding_[:, 0] % cylinder_dimension
h_coords = cylinder_mapper.embedding_[:, 1]
h_coords = H_pano * (h_coords - np.min(h_coords)) / (np.max(h_coords) - np.min(h_coords)) # Remap height to [0, H_pano] size of the Panorama

# Convert cylindrical coordinates to Cartesian coordinates
x = radius * np.cos(theta_coords)
y = radius * np.sin(theta_coords)
z = h_coords

embedding_cartesian = np.stack([x, y, z], axis=1)

fig = plt.figure(figsize=(20, 8))
ax1 = fig.add_subplot(121, projection='3d')
ax1.scatter(x, y, z, c=colors, s = 0.1)
ax1.set_title("Cylindrical projection", fontweight = "bold")

ax2 = fig.add_subplot(122)
ax2.scatter(theta_coords, h_coords, c=colors, s = 0.5)
ax2.set_title("Unwrapped cylinder", fontweight = "bold")

plt.tight_layout()
plt.show()

In [ ]:
# Create an entry in the map_projection_feature table for each feature, links features, media and coordinates
for i, row in df.iterrows():
    create_map_projection_feature(MapProjectionFeatureCreate(
        projection_id=projection_id,
        media_id=row.media_id,
        atlas_order=-1,
        index_in_atlas=-1,
        coordinates=[embedding_cartesian[i, 0], embedding_cartesian[i, 1], embedding_cartesian[i, 2]],
        feature_id=row.feature_id
    ))

In [ ]:
projection_id